# Barrier Option Pricing using Crank-Nicolson PDE Method

## Objective:
Implement and validate pricing of **Barrier Options** using a finite difference Crank-Nicolson method.

We focus on:
- Down-and-Out European Put option
- Barrier grid logic (zero value below barrier)
- Comparison with standard vanilla option

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

In [5]:
class CrankNicolsonBarrierSolver:
    def __init__(self, S_max, K, B, T, r, sigma, M=100, N=100, is_call=False, rebate=0.0):
        self.S_max = S_max
        self.K = K
        self.B = B
        self.T = T
        self.r = r
        self.sigma = sigma
        self.M = M  # time steps
        self.N = N  # asset steps
        self.is_call = is_call
        self.rebate = rebate
        
        self.dS = (S_max - B) / N
        self.dt = T / M
        self.grid = np.zeros((N + 1, M + 1))
        self.S = np.linspace(B, S_max, N + 1)
    
    def payoff(self, S):
        if self.is_call:
            return np.maximum(S - self.K, 0)
        else:
            return np.maximum(self.K - S, 0)
    
    def solve(self):
        i_vals = np.arange(1, self.N)
        dt, dS = self.dt, self.dS
        S = self.S
        
        alpha = 0.25 * dt * (self.sigma ** 2 * (i_vals ** 2) - self.r * i_vals)
        beta = -0.5 * dt * (self.sigma ** 2 * (i_vals ** 2) + self.r)
        gamma = 0.25 * dt * (self.sigma ** 2 * (i_vals ** 2) + self.r * i_vals)
        
        # Terminal condition
        self.grid[:, -1] = self.payoff(S)
        
        # Apply boundary condition: Knocked out below barrier
        self.grid[0, :] = self.rebate
        self.grid[-1, :] = self.payoff(S[-1]) if not self.is_call else 0
        
        M = self.M
        for j in reversed(range(M)):
            A = np.zeros((self.N - 1, self.N - 1))
            B = np.zeros(self.N - 1)
            
            for i in range(1, self.N):
                if i > 1:
                    A[i - 1, i - 2] = -alpha[i - 1]
                A[i - 1, i - 1] = 1 - beta[i - 1]
                if i < self.N - 1:
                    A[i - 1, i] = -gamma[i - 1]
                
                B[i - 1] = alpha[i - 1] * self.grid[i - 1, j + 1] + \
                           (1 + beta[i - 1]) * self.grid[i, j + 1] + \
                           gamma[i - 1] * self.grid[i + 1, j + 1]
            
            x = np.linalg.solve(A, B)
            self.grid[1:self.N, j] = x
        
        return np.interp(self.K, S, self.grid[:, 0])  # Option price at S = K

In [6]:
# Parameters
S_max = 200
B = 80        # Barrier (below S0)
K = 100
T = 1.0
r = 0.05
sigma = 0.2
S0 = 100

solver = CrankNicolsonBarrierSolver(S_max=S_max, K=K, B=B, T=T, r=r, sigma=sigma, is_call=False)
barrier_price = solver.solve()
print(f"Down-and-Out Put Option Price (PDE): {barrier_price:.4f}")

Down-and-Out Put Option Price (PDE): 1.1273


## Summary:

We implemented a finite difference Crank-Nicolson scheme to price a **Down-and-Out European Put**.

### Key Points:
- **Barrier Enforcement:** Grid values below the barrier are set to zero (knock-out condition).
- **Convergence:** The result can be verified by increasing grid resolution.
- **Comparison:** To be validated against Monte Carlo results (see next notebook).

This method shows how PDEs can handle barrier conditions effectively with proper grid alignment and boundary handling.